In [1]:
# Fusion ART Model for Professor Data
# Python fusionART.py code by Dr. Budhitama Subagdja, modified for this dataset by Patrick Tjahjadi

# ----- Imported Libraries -----
from fusionART import *

from collections import defaultdict

import pandas as pd

import math

import random

from numpy import array
from numpy import argmax

import numpy as np

from itertools import islice

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 

import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import gensim.downloader as api
from gensim import corpora

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import csv

import wikipedia

import dill
# -------------------------------

data = pd.read_csv('SCSE ProfProfile.csv', encoding='ISO-8859-1')

In [2]:
# Function to convert 1D lists into 2D, based on the number of elements per list.
def convert_to_2d(list_to_convert, list_per_entry): 
    it = iter(list_to_convert) 
    return [list(islice(it, i)) for i in list_per_entry] 

In [3]:
# Convert each attribute of the dataset to separate lists
name = []
group = []
university = []
research_interest = []
for idx, row in data.iterrows():
    name.append(row[0])
    group.append(row[1])
    university.append(row[2])
    research_interest.append(row[3])

# Problem: research_interest and university uses semicolons to denote multiple interests and universities
# Change their format to a list of universities and research interests of a person

university_processed = []
for entry in university:
    uni = entry.split(";")
    university_processed.append(uni)

research_interest_preprocessed = []
for entry in research_interest:
    interests = entry.split(";")
    research_interest_preprocessed.append(interests)    

    
# Break university_processed and research_interest_preprocessed to 1D lists for one hot encoding
university_1d = []
for universities in university_processed:
    for uni in universities:
        university_1d.append(uni)
        
research_interest_1d = []
for interests in research_interest_preprocessed:
    for interest in interests:
        research_interest_1d.append(interest)
        


In [4]:
# Determine the number of universities and research interests per professor

universities_per_person = []
for universities in university_processed:
    universities_per_person.append(len(universities))
    

research_interest_per_person = []
for research_interests in research_interest_preprocessed:
    research_interest_per_person.append(len(research_interests))


In [5]:
# Research interest preprocessing, to consider each research keywords as a phrase
for index in range(0, len(research_interest_1d)):
    research_interest_1d[index] = research_interest_1d[index].replace(" ", "_")
    research_interest_1d[index] = research_interest_1d[index].lower()
    research_interest_1d[index] = research_interest_1d[index].replace(":", "")
    research_interest_1d[index] = research_interest_1d[index].replace(",", "")


In [6]:
# One Hot Encoding for name, group, university and research interests
name_label_encoder = LabelEncoder()
uni_label_encoder = LabelEncoder()
group_label_encoder = LabelEncoder()
research_label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder(sparse=False)

name_integer_encoded = name_label_encoder.fit_transform(name)
name_integer_encoded = name_integer_encoded.reshape(len(name_integer_encoded), 1)
name_onehot = onehot_encoder.fit_transform(name_integer_encoded)

group_integer_encoded = group_label_encoder.fit_transform(group)
group_integer_encoded = group_integer_encoded.reshape(len(group_integer_encoded), 1)
group_onehot = onehot_encoder.fit_transform(group_integer_encoded)

university_integer_encoded = uni_label_encoder.fit_transform(array(university_1d))
university_integer_encoded = university_integer_encoded.reshape(len(university_integer_encoded), 1)
university_onehot = onehot_encoder.fit_transform(university_integer_encoded)

research_interest_integer_encoded = research_label_encoder.fit_transform(array(research_interest_1d))
research_interest_integer_encoded = research_interest_integer_encoded.reshape(len(research_interest_integer_encoded), 1)
research_interest_onehot = onehot_encoder.fit_transform(research_interest_integer_encoded)


In [7]:
# Convert the numpy arrays to lists
name_onehotlist = name_onehot.tolist()
group_onehotlist = group_onehot.tolist()
university_onehotlist = university_onehot.tolist()
research_interest_onehotlist = research_interest_onehot.tolist()


In [8]:
# Perform the one-hot research keywords noise flipping here

def one_hot_keyword_flip(query_research, noise_limit):
    
    query_research = query_research.split(";")
    
    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")

    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research)):
        for onehot_vec in research_interest_onehotlist:
            if (str(query_research[research_index]).rstrip("\n") == 
                research_label_encoder.inverse_transform([argmax(onehot_vec)])[0]):
                query_research[research_index] = onehot_vec
                

        
    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if type(query_research[index]) != list:
            del query_research[index]
        else:
            index += 1


    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]
    
    # With a defined probability, flip the binary research keyword vectors due to noise
    for index in range(0, len(research_vector)):
        noise_value = random.random()
        if (noise_value < noise_limit):
            if (research_vector[index] == 0):
                research_vector[index] = 1
            else:
                research_vector[index] = 0
    return research_vector


In [9]:
# Research keywords flip for 0% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_zero_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [10]:
# Research keywords flip for 10% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.1)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_ten_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)


In [11]:
# Research keywords flip for 20% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.2)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_twenty_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [12]:
# Research keywords flip for 30% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.3)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_thirty_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [13]:
# Research keywords flip for 40% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.4)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_forty_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [14]:
# Research keywords flip for 50% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.5)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_fifty_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [18]:
"""
# Create the new input text file from the noisy binary keyword vector
output_file = open("noise_ten_research_query.txt", "w+")
index = 0
first = 1
for professor in name:
    output_file.write(professor+"\t")
    for keyword in range(sum(noisy_research_interest_per_person[0:index]), sum(noisy_research_interest_per_person[0:index+1])):
        output_file.write(noisy_research_interest_1d[keyword])
        if keyword != (sum(noisy_research_interest_per_person[:index+1]) - 1):
            output_file.write(";")
        
    index += 1
    output_file.write("\n")
"""

'\n# Create the new input text file from the noisy binary keyword vector\noutput_file = open("noise_ten_research_query.txt", "w+")\nindex = 0\nfirst = 1\nfor professor in name:\n    output_file.write(professor+"\t")\n    for keyword in range(sum(noisy_research_interest_per_person[0:index]), sum(noisy_research_interest_per_person[0:index+1])):\n        output_file.write(noisy_research_interest_1d[keyword])\n        if keyword != (sum(noisy_research_interest_per_person[:index+1]) - 1):\n            output_file.write(";")\n        \n    index += 1\n    output_file.write("\n")\n'

In [15]:
# Create a list of unique names, groups, universities and research interests to define the model schema
unique_name = []
unique_group = []
unique_university = []
unique_research_interest = []
[unique_name.append(i) for i in name if i not in unique_name]
[unique_group.append(i) for i in group if i not in unique_group]
[unique_university.append(i) for i in university_1d if i not in unique_university]
[unique_research_interest.append(i) for i in research_interest_1d if i not in unique_research_interest]
pass

In [16]:
# Problem: each professor has multiple universities and research interests
# Convert these 1D arrays to 2D, to associate them to each professor

university_onehotlist_processed = []
research_interest_onehotlist_processed = []

university_onehotlist_processed = convert_to_2d(university_onehotlist, universities_per_person)
research_interest_processed = convert_to_2d(research_interest_onehotlist, research_interest_per_person)

In [17]:
# Create an array that allows multiple universities and research interests per professor by summation
university_2d = []

for universities_per_professor in university_onehotlist_processed:
    university_2d.append([sum(x) for x in zip(*universities_per_professor)])

    
research_interest_2d = []
for research_interests_per_professor in research_interest_processed:
    research_interest_2d.append([sum(x) for x in zip(*research_interests_per_professor)])

# Since universities can be duplicate, change so that the maximum value of each university is 1
for universities_per_professor in university_2d:
    for index in range(0, len(universities_per_professor)):
        if universities_per_professor[index] > 1:
            universities_per_professor[index] = 1.0


In [18]:
# Define the schema for the FusionART model
model_schema = [{'name': data.columns[0], 'compl': False, 'attrib': sorted(unique_name)}, 
                {'name': data.columns[1], 'compl': False, 'attrib': sorted(unique_group)},
                {'name': data.columns[2], 'compl': False, 'attrib': sorted(unique_university)},
                {'name': data.columns[3], 'compl': True, 'attrib': sorted(unique_research_interest)}]

In [19]:
# Initialise the FusionART model
model = FusionART(schema = model_schema, beta = [1.0, 1.0, 1.0, 1.0], alpha = [0.1, 0.1, 0.1, 0.1], 
               gamma = [0.25, 0.25, 0.25, 0.25], rho = [1, 1, 1, 1])
model.F1Fields

[{'name': 'name',
  'compl': False,
  'attrib': ['AS Madhukumar',
   'Alexei Sourin',
   'Anupam Chattopadhyay',
   'Anwitaman Datta',
   'Arijit Khan',
   'Arvind Easwaran',
   'Bo An',
   'Cai Jianfei',
   'Cai Wentong',
   'Cham Tat Jen',
   'Chan Syin',
   'Chia Liang Tien Clement',
   'Chng Eng Siong',
   'Deepu Rajan',
   'Deng Ruilong',
   'Douglas Leslie Maskell',
   'Dusit Niyato',
   'Eric Cambria',
   'Gao Cong',
   'Goh Wooi Boon',
   'Guan Cuntai',
   'He Ying',
   'Huang Shell Ying',
   'Hui Siu Cheung',
   'Jagath C. Rajapakse',
   'Ke Yiping Kelly',
   'Kong Wai Kin Adams',
   'Kwoh Chee Keong',
   'Lam Kwok Yan',
   'Lam Siew Kei',
   'Lau Chiew Tong',
   'Lee Bu Sung Francis',
   'Li Fang Flora',
   'Li Mo',
   'Li Yi',
   'Liang Qianhui Althea',
   'Lin Feng',
   'Lin Guosheng',
   'Lin Shang Wei',
   'Lin Weisi',
   'Liu Weichen',
   'Liu Yang',
   'Loke Yuan Ren',
   'Long Cheng',
   'Loy Chen Change (Cavan)',
   'Lu Shijian',
   'Luo Jun',
   'Mahardhika Pratama',

In [20]:
# Store the data to the Fusion ART model

for i in range(0, len(name)):
    model.updateF1bySchema([{'name': data.columns[0], 'val': name_onehotlist[i]}, 
                            {'name': data.columns[1], 'val': group_onehotlist[i]}, 
                            {'name': data.columns[2], 'val': university_2d[i]}, 
                            {'name': data.columns[3], 'val': research_interest_2d[i]}])

    
    print("resonance search: ")
    J = model.resSearch()
    print("selected ", J)
    if model.uncommitted(J):
        print ('uncommitted')

    model.autoLearn(J)
    model.doReadoutAllFields(J)
                    
    
model.displayNetwork()

resonance search: 
selected  0
uncommitted
resonance search: 
selected  1
uncommitted
resonance search: 
selected  2
uncommitted
resonance search: 
selected  3
uncommitted
resonance search: 
selected  4
uncommitted
resonance search: 
selected  5
uncommitted
resonance search: 
selected  6
uncommitted
resonance search: 
selected  7
uncommitted
resonance search: 
selected  8
uncommitted
resonance search: 
selected  9
uncommitted
resonance search: 
selected  10
uncommitted
resonance search: 
selected  11
uncommitted
resonance search: 
selected  12
uncommitted
resonance search: 
selected  13
uncommitted
resonance search: 
selected  14
uncommitted
resonance search: 
selected  15
uncommitted
resonance search: 
selected  16
uncommitted
resonance search: 
selected  17
uncommitted
resonance search: 
selected  18
uncommitted
resonance search: 
selected  19
uncommitted
resonance search: 
selected  20
uncommitted
resonance search: 
selected  21
uncommitted
resonance search: 
selected  22
uncommitte

In [21]:
# Perform the research keywords query here

def query_by_research_with_noise(query_research, output_file):
    
    query_research = query_research.split(";")
    output_file.write("Research Keywords: ")
    output_file.write(', '.join(query_research)+"\n")
    
    
    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")
    
    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research)):
        for onehot_vec in research_interest_onehotlist:
            if (str(query_research[research_index]).rstrip("\n") == 
                research_label_encoder.inverse_transform([argmax(onehot_vec)])[0]):
                query_research[research_index] = onehot_vec
    
    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if (type(query_research[index]) != list or query_research[index] == []):
            del query_research[index]
        else:
            index += 1
    
    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]            
    
    
    # Define the complement-coded research vector
    compl_vector = []
    for bit in research_vector:
        compl_vector.append(1 - bit)
        
    model.setParam('gamma', [0,0,0,1])
    model.setParam('rho', [0,0,0,0])
    model.updateF1bySchema([{'name': data.columns[3], 'val': research_vector, 'vcompl': compl_vector}], refresh = False)
    model.compChoice()

    # Find the nodes that have the highest F2 value
    maxF2value = model.codes[0]['F2']
    maxF2indexes = []

    for i in range(0, len(model.codes)-1):
        F2value = model.codes[i]['F2']
        if (F2value > maxF2value):
            maxF2value = F2value
            maxF2indexes = [i]
        elif (F2value == maxF2value):
            maxF2indexes.append(i)

    # Retrieve the data of the node(s) that have the highest F2 value
    output_file.write("Professor that follows these research keywords:")    
    for node in maxF2indexes:
        name_vector = model.codes[node]['weights'][0]
        for index in range(0, len(name_vector)):
            # Iterate through the vector and find the indexes that has a value of 1
            if (name_vector[index] == 1):
                # Retrieve the data from the model's 'attrib' attribute in F1 
                retrieved_name = model.F1Fields[0]['attrib'][index]
                output_file.write(retrieved_name)
    
    output_file.write("\n\n")
    return(retrieved_name)


In [22]:
# Research keywords query for 0% noise
output_file = open("output_zero_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_zero_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()


In [23]:
# Research keywords query for 10% noise
output_file = open("output_ten_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_ten_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [24]:
# Research keywords query for 20% noise
output_file = open("output_twenty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_twenty_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [25]:
# Research keywords query for 30% noise
output_file = open("output_thirty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_thirty_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [26]:
# Research keywords query for 40% noise
output_file = open("output_forty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_forty_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [27]:
# Research keywords query for 50% noise
output_file = open("output_fifty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_fifty_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

In [52]:
output_file.close()

